### Translation for kor → eng

o 연구개발명 : 한국어-영어 번역 모델 개발

o 연구개발내용(요약) : 번역 모델 개발을 목적으로 학습하려고 함. 개인적인 공부 목적임.

o 연구개발기간(예상) : 20년 3월 ~ 20년 12월

In [1]:
import sys
sys.path.insert(0, '../attention-is-all-you-need/')
sys.path.insert(0, '../')

In [4]:
import sentencepiece as spm
import random
import pandas as pd
import os
import numpy as np
from setting import *
from dataset import load_dataset

In [11]:
sent_pairs = []
for f in os.listdir('../attention-is-all-you-need/data/'):
    one_df = pd.read_excel(os.path.join('../attention-is-all-you-need/data', f))
    one_df = one_df.rename(columns={
        '영어':'eng',
        '한국어':'kor',
        '원문':'kor',
        '영어 검수':'label',
        '영어검수':'label',
        'Review':'label',
        'REVIEW':'label',
    })
    print(f, one_df.columns)
    sent_pairs.extend(one_df[['kor','label']].values.tolist())

2.xlsx Index(['대분류', '소분류', '상황', 'Set Nr.', '발화자', 'kor', 'eng', 'label'], dtype='object')
4.xlsx Index(['ID', '키워드', 'kor', 'NMT', 'PE', 'label'], dtype='object')
1.xlsx Index(['Temp', 'kor', 'eng', 'label'], dtype='object')
3.xlsx Index(['ID', '날짜', '자동분류_1', '자동분류_2', '자동분류_3', 'URL', '언론사', 'kor', 'NMT',
       'PE', 'label'],
      dtype='object')
5.xlsx Index(['ID', '키워드', 'kor', 'NMT', 'PE', 'label'], dtype='object')
6.xlsx Index(['ID', '지자체', '제목', 'URL', 'kor', 'NMT', 'PE', 'label'], dtype='object')


In [23]:
with open('./spm_src.txt', 'w', encoding='utf-8') as f:
    for sent in sent_pairs:
        f.write('{}\n'.format(sent[0]))

In [24]:
with open('./spm_trg.txt', 'w', encoding='utf-8') as f:
    for sent in sent_pairs:
        f.write('{}\n'.format(sent[1]))

In [29]:
! wc -l ./spm_*.txt

   66262 ./spm_src.txt
   66263 ./spm_trg.txt
  132525 total


In [30]:
! head -n 3 ./spm_src.txt

다른 학교 친구들을 데려가도 될까요?
부산미곡증권신탁회사는 조선총독부가 1930년대 초반 추진한 ‘신탁 합동’에 의해 1931년 해체되었다.
“수탁자”라 함은 제6조제1항에 의하여 시설의 관리·운영을 위탁받은 법인·단체 또는 개인 등을 말한다.


In [31]:
! head -n 3 ./spm_trg.txt

May I bring friends from another school?
Busan Migok Trust Company closed its business in 1931 due to the "Trust Cooperation" promoted by Japanese Government General of Korea in the early 1930s.
The term "consignee" means a corporation, organization, individual, etc. entrusted with the management and operation of facilities pursuant to Article 6 (1).


In [32]:
random.shuffle(sent_pairs)

In [33]:
sent_pairs[0]

['다음 주부터 시험 기간이라 동아리 활동은 없어요. 모두 공부 열심히 해요.',
 'There will be no club activity next week because of the exam. Study hard, everyone.']

In [34]:
src_vocab_size = 20000  # 50000
trg_vocab_size = 16000  # 32000
src_prefix = 'spm-src-{}'.format(src_vocab_size)
trg_prefix = 'spm-trg-{}'.format(trg_vocab_size)
src_prefix, trg_prefix

('spm-src-20000', 'spm-trg-16000')

In [35]:
src_cmd = templates.format(
    src_input_file,
    pad_id,
    bos_id,
    eos_id,
    unk_id,
    src_prefix,
    src_vocab_size,
    character_coverage,
    model_type)
src_cmd

'--input=./spm_src.txt --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3 --model_prefix=spm-src-20000 --vocab_size=20000 --character_coverage=1.0 --model_type=unigram'

In [36]:
trg_cmd = templates.format(
    trg_input_file,
    pad_id,
    bos_id,
    eos_id,
    unk_id,
    trg_prefix,
    trg_vocab_size,
    character_coverage,
    model_type)
trg_cmd

'--input=./spm_trg.txt --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3 --model_prefix=spm-trg-16000 --vocab_size=16000 --character_coverage=1.0 --model_type=unigram'

In [37]:
spm.SentencePieceTrainer.Train(src_cmd)
spm.SentencePieceTrainer.Train(trg_cmd)

True

In [41]:
! ls -alh *.model

-rw-r--r-- 1 jkfirst deep-learners 605K  3월 25 13:20 spm-src-20000.model
-rw-r--r-- 1 jkfirst deep-learners 508K  3월 25 13:20 spm-trg-16000.model


In [43]:
src_spm = spm.SentencePieceProcessor()
trg_spm = spm.SentencePieceProcessor()
src_spm.Load('{}.model'.format(src_prefix)) 
trg_spm.Load('{}.model'.format(trg_prefix))

True

In [44]:
with open('{}.vocab'.format(src_prefix), encoding='utf-8') as f:
    src_vocab = [doc.strip() for doc in f]
src_vocab[:10]

['<pad>\t0',
 '<s>\t0',
 '</s>\t0',
 '<unk>\t0',
 '.\t-3.26792',
 '을\t-3.6955',
 '의\t-3.95254',
 '를\t-4.15871',
 '▁\t-4.16494',
 ',\t-4.24634']

In [45]:
with open('{}.vocab'.format(trg_prefix), encoding='utf-8') as f:
    trg_vocab = [doc.strip() for doc in f]
trg_vocab[:10]

['<pad>\t0',
 '<s>\t0',
 '</s>\t0',
 '<unk>\t0',
 '▁the\t-2.7476',
 ',\t-3.29956',
 '▁of\t-3.32764',
 '.\t-3.49639',
 '▁and\t-3.78203',
 '▁to\t-3.79164']

In [46]:
idx = np.random.randint(0, len(sent_pairs))
idx
src_spm.EncodeAsPieces(sent_pairs[idx][0])

['▁“',
 '이',
 '거',
 '▁오래',
 '▁입을',
 '▁수',
 '▁있을까',
 '”',
 '▁“',
 '얼',
 '마',
 '나',
 '▁자주',
 '▁입을',
 '까',
 '”',
 '▁“',
 '옷',
 '이',
 '▁투자',
 '할',
 '▁만한',
 '▁가치',
 '가',
 '▁있는',
 '가',
 '”',
 '▁하는',
 '▁질문',
 '입니다',
 '.']

In [47]:
idx = np.random.randint(0, len(sent_pairs))
trg_spm.EncodeAsPieces(sent_pairs[idx][1])

['▁Thank',
 '▁you',
 '▁for',
 '▁your',
 '▁interest',
 '▁in',
 '▁our',
 '▁shutt',
 'le',
 '▁service',
 '.',
 '▁What',
 '▁might',
 '▁the',
 '▁matter',
 '▁be',
 '?']

In [48]:
idx = np.random.randint(0, len(sent_pairs))
src_sent = trg_spm.EncodeAsPieces(sent_pairs[idx][0])
trg_sent = trg_spm.EncodeAsPieces(sent_pairs[idx][1])
src_sent, trg_sent

(['▁“',
  '정',
  '보',
  '주',
  '체',
  '”',
  '란',
  '▁',
  '영',
  '상',
  '정',
  '보',
  '에',
  '▁',
  '의',
  '하',
  '여',
  '▁',
  '식',
  '별',
  '되',
  '는',
  '▁',
  '사',
  '람',
  '으',
  '로',
  '서',
  '▁',
  '해',
  '당',
  '▁',
  '영',
  '상',
  '정',
  '보',
  '의',
  '▁',
  '주',
  '체',
  '가',
  '▁',
  '되',
  '는',
  '▁',
  '자',
  '연',
  '인',
  '을',
  '▁',
  '말',
  '한',
  '다',
  '.'],
 ['▁The',
  '▁term',
  '▁"',
  'information',
  '▁object',
  '"',
  '▁means',
  '▁a',
  '▁person',
  '▁identified',
  '▁by',
  '▁video',
  '▁information',
  ',',
  '▁who',
  '▁is',
  '▁the',
  '▁subject',
  '▁of',
  '▁the',
  '▁relevan',
  't',
  '▁video',
  '▁information',
  '.'])